In [ ]:
import numpy as np
import gtsam
from gtsam.symbol_shorthand import X, P
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
from sln_letter_fit import FitParams
from sln_stroke_fit import SlnStrokeFit
from art_skills import SlnStrokeExpression
import loader
from fit_types import Solution

%load_ext autoreload
%autoreload 1
%aimport chebyshev_fitter, spline_fitter, sln_letter_fit, loader, sln_stroke_fit

# First Pretend we already have the stroke splits

In [ ]:
letter = loader.load_segments('D')
dt = 1./120
fit_params = FitParams()
fitter = SlnStrokeFit(dt,
                      integration_noise_model=gtsam.noiseModel.Isotropic.Sigma(2, fit_params.noise_integration_std),
                      data_prior_noise_model=gtsam.noiseModel.Isotropic.Sigma(2, fit_params.noise_data_prior_std),
                      reparameterize=fit_params.reparameterize,
                      flip_parameters_at_end=fit_params.flip_parameters_at_end)
stroke_indices = fitter.stroke_indices(letter)
stroke_param_mean, stroke_param_prior = (np.array([
    -0.3824788860477358, 0.4333073800807866, 0.3254905871611055, 0.20628570778082297,
    0.23762343512462863, -1.104228059293241
]), gtsam.noiseModel.Isotropic.Sigma(6, 50))
isam2params = gtsam.ISAM2Params()
isam2params.evaluateNonlinearError = True


In [ ]:
class IncrementalStrokeFit:

    def __init__(self,
                 fitter,
                 kstart,
                 stroke_index,
                 isam2params=gtsam.ISAM2Params(),
                 stroke_params_init=stroke_param_mean,
                 progress_bar = None):
        self.fitter = fitter
        self.k = self.kstart = kstart
        self.stroke_index = stroke_index
        self.progress_bar = progress_bar
        # initialize isam
        self.isam = gtsam.ISAM2(isam2params)
        graph = gtsam.NonlinearFactorGraph()
        graph.addPriorVector(P(stroke_index), stroke_param_mean, stroke_param_prior)
        init = gtsam.Values()
        init.insert(P(strokei), stroke_params_init)
        result = self.isam.update(graph, init)
        self.error = result.getErrorAfter()

    def add_datapoint(self, txy):
        t, x, y = txy
        k = self.fitter.t2k(t)
        assert k == self.k, f'Time mismatch: {k = }\t{self.k = }'

        # New factors
        graph = gtsam.NonlinearFactorGraph()
        graph.push_back(self.fitter.data_prior_factors(np.array([[t, x, y]])))
        if k > self.kstart:
            graph.push_back(self.fitter.stroke_factors(self.stroke_index, k - 1, k))
        # New variables
        init = gtsam.Values()
        # if k == 0:
        init.insert(X(k), np.array([x, y]))
        # else:
        #     stroke_params = self.isam.calculateEstimateVector(P(self.stroke_index))
        #     stroke_eval = SlnStrokeExpression(stroke_params)
        #     xy_init = self.isam.calculateEstimatePoint2(X(k-1)) + stroke_eval.displacement(t, dt)
        #     init.insert(X(k), xy_init)

        # incremental solve
        result = self.isam.update(graph, init)

        for _ in range(20):
            if self.check_convergence(result):
                break
            result = self.isam.update()

        self.error = result.getErrorAfter()
        self.k += 1
        self.progress_bar.update()
        return self.isam.calculateEstimate()

    def check_convergence(self, result):
        k = self.k
        if self.progress_bar is not None:
            b, a = max(self.error, 1e-8), result.getErrorAfter()
            if True or a != did_update:
                # did_update = a
                adjCost = (a * k / (k + 1) - b) / b if k > 0 else float('nan')
                self.progress_bar.set_postfix(errorBefore=f'{result.getErrorBefore():.2e}',
                                         errorAfter=f'{result.getErrorAfter():.2e}',
                                         adjustedCostIncrease=f'{adjCost:.3f}')
        if self.error == 0:
            return True
        return (result.getErrorAfter() * (k-1) / k - self.error) / self.error < 1e-2

In [ ]:
letter_estimates = []
with tqdm.notebook.trange(sum(stroke.shape[0] for stroke in letter)) as progress_bar:
    progress_bar.set_description('Optimizing stroke')
    for strokei, stroke in enumerate(letter):
        iSF = IncrementalStrokeFit(fitter,
                                   stroke_indices[strokei][0],
                                   strokei,
                                   isam2params=isam2params,
                                   stroke_params_init=stroke_param_mean,
                                   progress_bar=progress_bar)
        letter_estimates.append([])
        for datai, txy in enumerate(stroke):
            letter_estimates[-1].append(iSF.add_datapoint(txy))

In [ ]:
new_estimates = []
for strokei, stroke_estimates in enumerate(letter_estimates):
    for estimate in stroke_estimates:
        new_estimates.append(gtsam.Values(estimate))
        for prev_stroke_estimates in letter_estimates[:strokei]:
            new_estimates[-1].insert(prev_stroke_estimates[-1])

def create_sol(values, k_end):
    t = np.arange(0, k_end + 1).reshape(-1, 1) * dt
    x0 = fitter.query_estimate_at(values, 0)
    def query(t):
        try:
            return fitter.query_estimate_at(values, t)
        except:
            return np.zeros(2)#x0
    txy = np.hstack((t, [query(t_) for t_ in t]))
    num_strokes = sum(1 for start, _ in stroke_indices.values() if k_end > start)
    params = [fitter.query_parameters(values, strokei) for strokei in range(num_strokes)]
    txy_from_params = fitter.compute_trajectory_from_parameters(x0, params, stroke_indices)
    txy_from_params = np.hstack((np.arange(0, txy_from_params.shape[0] * dt,
                                           dt).reshape(-1, 1), txy_from_params))
    return Solution(params=params,
                    txy=txy,
                    txy_from_params=txy_from_params,
                    stroke_indices=stroke_indices)


history = [create_sol(estimate, estimatei) for estimatei, estimate in enumerate(new_estimates)]
sol_and_history = (history[-1], history)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

txy_gt = np.vstack(letter)
def update(i):
    ax.cla()
    ax.plot(txy_gt[:i + 1, 1], txy_gt[:i + 1, 2], 'k.')
    est = history[i]
    ax.plot(est['txy'][:i + 1, 1], est['txy'][:i + 1, 2], 'r.', markersize=1)
    ax.plot(est['txy_from_params'][:i + 1, 1],
            est['txy_from_params'][:i + 1, 2],
            'r-',
            markersize=1)
    ax.plot(est['txy_from_params'][i:i + 10, 1],
            est['txy_from_params'][i:i + 10, 2],
            'b-',
            markersize=1)

    text = '\n'.join(f'params: ' + ''.join(f'{param:6.2f}'.replace(' ', '~')
                                           for param in params)
                     for params in history[i]['params'])
    text_box = matplotlib.offsetbox.AnchoredText(text,
                                                 frameon=True,
                                                 loc='lower right',
                                                 pad=0.3,
                                                 bbox_to_anchor=(1, 0),
                                                 bbox_transform=ax.transAxes,
                                                 borderpad=0,
                                                 prop=dict(size=9))
    text_box.patch.set_alpha(0.4)
    ax.add_artist(text_box)

    ax.axis('equal')
    ax.set_ylim(-1.1, -0.1)
    ax.set_xlim(0, 1.05)

# with tqdm.notebook.trange(0, len(letter[0]), 1) as progress_bar:
#     progress_bar.set_description('Saving Animation')
#     anim = matplotlib.animation.FuncAnimation(ax.figure, update, frames=progress_bar)
#     anim.save('results/incremental_D_stroke0.mp4', writer=matplotlib.animation.FFMpegWriter(fps=15))
# anim = plotting.animate_trajectories(ax, [letter], [sol_and_history], is_notebook=True)
with tqdm.notebook.trange(0, len(history), 1) as progress_bar:
    progress_bar.set_description('Displaying Animation')
    anim = matplotlib.animation.FuncAnimation(ax.figure, update, frames=progress_bar)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))
